In [16]:
import pandas as pd
import numpy as np
from scipy import stats
import pingouin as pg
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [6]:
df = pd.read_csv("/Users/beans/Desktop/Mod-4-Project/data/hour.csv")

df['season'] = df['season'].replace(1,'winter')
df['season'] = df['season'].replace(2,'spring')
df['season'] = df['season'].replace(3,'summer')
df['season'] = df['season'].replace(4,'fall')


In [ ]:
# STATISTICAL ANALYSIS

# Task 1

# Do average hourly rides differ between working days and non-working days?

# H 0 : Average hourly working day rides do not differ from average hourly non-working day rides.

# H 1 : Average hourly working day rides differ from average hourly non-working day rides.

#filtering to get averge rides for both non-working and workingdays
workingday = df[df['workingday'] == 1]
notworkingday = df[df['workingday'] == 0]

workingmean = workingday.groupby('hr')['cnt'].mean().reset_index()
notworkingmean = notworkingday.groupby('hr')['cnt'].mean().reset_index()


# stats ttest
test = stats.ttest_ind(workingmean['cnt'],notworkingmean['cnt'], equal_var=False,alternative='two-sided')

test

# pinguoin ttest

ping = pg.ttest(x=workingmean['cnt'],y=notworkingmean['cnt'],confidence=0.05)
ping

# BOTH TESTS SAY THERE IS NO SIGNIFICANT DIFFERENCE BETWEEN THE RIDER AVERAGES OF WORKING DAYS AND NOT WORKING DAYS

,hr,cnt
0,0,36.786290
1,1,16.552632
2,2,8.683778
3,3,4.942553
4,4,5.429787
5,5,24.913131
6,6,102.500000
7,7,290.612903
8,8,477.006048
9,9,241.518145


Our two-sided two sample ttest indicates there is no statistically significant difference between hourly working day rides and hourly non-working day rides, thus we fail to reject the Null Hypothesis.

In [55]:
# Task 2

# Do mean hourly rides differ across categories of multi-level categorical variables such as
# season or weather condition (choose one)? If you find a difference, describe the appropriate
# post-hoc (after the test what other tests would you do) approach and what it would tell
# stakeholders.

# H 0 : There is no difference between hourly ride means across seasons.

# H 1 : There is a difference between houry ride means across seasons.

alpha = 0.05

#filtering to separate seasons
winter = df[df['season'] == 'winter']
spring = df[df['season'] == 'spring']
summer = df[df['season'] == 'summer']
fall = df[df['season'] == 'fall']

wintermean = winter.groupby('hr')['cnt'].mean().reset_index()
springmean = spring.groupby('hr')['cnt'].mean().reset_index()
summermean = summer.groupby('hr')['cnt'].mean().reset_index()
fallmean = fall.groupby('hr')['cnt'].mean().reset_index()

wintermean = wintermean.rename(columns={'cnt':'wintercount'})
springmean = springmean.rename(columns={'cnt':'springcount'})
summermean = summermean.rename(columns={'cnt':'summercount'})
fallmean = fallmean.rename(columns={'cnt':'fallcount'})

#merging all seasons to avoid any issues when reshaping dataframe
swseasons = pd.merge(wintermean,springmean,on='hr',how='left')
sfseasons = pd.merge(summermean,fallmean,on='hr',how='left')

fullseasons = pd.merge(swseasons,sfseasons,on='hr',how='left')
fullseasons = fullseasons.drop('hr',axis=1)

# ANOVA TEST

f_statistic, p_value = f_oneway(fullseasons['wintercount'],fullseasons['springcount'],fullseasons['summercount'],fullseasons['fallcount'])

print('P-Value is:',p_value)
print('F-statistic is :',f_statistic)
# the p_value shows at least one group's mean is statistically different from the others

# RESHAPING FOR TUKEY'S TEST

reshaped_seasons = pd.melt(fullseasons, value_vars = ['wintercount', 'springcount','summercount','fallcount'], var_name='seasons', value_name='mean_count')

tukey_results = pairwise_tukeyhsd(endog=reshaped_seasons['mean_count'], groups = reshaped_seasons['seasons'], alpha=alpha)

print(tukey_results)

P-Value is: 0.011211790379078128
F-statistic is : 3.9091771336107466
       Multiple Comparison of Means - Tukey HSD, FWER=0.05        
   group1      group2    meandiff p-adj    lower    upper   reject
------------------------------------------------------------------
  fallcount springcount    9.4842  0.995  -92.8854 111.8537  False
  fallcount summercount   37.1261 0.7785  -65.2434 139.4956  False
  fallcount wintercount   -89.223 0.1101 -191.5925  13.1465  False
springcount summercount   27.6419 0.8943  -74.7276 130.0114  False
springcount wintercount  -98.7072  0.063 -201.0767   3.6623  False
summercount wintercount -126.3491 0.0092 -228.7186 -23.9796   True
------------------------------------------------------------------


After conducting an ANOVA test to determine whether one of our categories' means is statistically different from the others, our p-value was less than our alpha. This would mean we decide to reject the Null Hypothesis and figure out there is a difference between at least one of our seasons' hourly ride mean. The next steps would be to perform a post-hoc test, such as Tukey's, to determine which combination of seasons has the statistically significant difference. After conducting our Tukey's test, I was able to find that our summer mean bike rider count is statistically different than our winter mean bike rider count, which makes sense as bike riding is far less popular over the winter than it is over the summer.